In [16]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as colors

import scipy.constants as c
from scipy import stats

from photutils.aperture import CircularAperture
from photutils.detection import DAOStarFinder
from photutils.aperture import aperture_photometry
from photutils.aperture import CircularAnnulus

from astropy.utils.data import get_pkg_data_filename
from astropy.wcs import WCS
from astropy import wcs
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy import units as u

import os
import sys
from astropy.table import Table, join, vstack
from astropy.nddata.utils import extract_array
import pyregion
import tqdm 
import argparse

import integratedflux as flux
import integratedflux_py3 as flux_py3

from importlib import reload
reload(flux)
import warnings
from astropy.utils.exceptions import AstropyWarning

import time
from regions import Regions

warnings.filterwarnings('ignore', category=AstropyWarning, append=True)

In [22]:
directory_univ_I = r'/net/vdesk/data2/GoesaertW/Meerkat_Data/Abell_85/I_slices/'
directory_univ_Q = r'/net/vdesk/data2/GoesaertW/Meerkat_Data/Abell_85/Q_slices/'
directory_univ_U = r'/net/vdesk/data2/GoesaertW/Meerkat_Data/Abell_85/U_slices/'

directory_local = r'/data2/goesaert/' 

region_name = "Abell85_catalog_BDSF_gaul.reg"

rsl = fits.open(get_pkg_data_filename(directory_local+'Abell85_catalog_BDSF_rsl.fits'))[1].data

In [28]:
hdul_image = fits.open('/net/vdesk/data2/GoesaertW/Meerkat_Data/Abell_85/I_slices/Abell_85_I_plane_freq01.fits')

head = hdul_image[0].header
data = hdul_image[0].data 
w  = wcs.WCS(head)

reg_file = Regions.read(region_name)
print(reg_file[0].to_pixel(w))

ValueError: Number of world inputs (1) does not match expected (3)

In [ ]:
warnings.filterwarnings('ignore', category=AstropyWarning, append=True)

flux, nbeams = integratedflux(fitsimage, ds9region, i, hdul=None, test=False)
mask = flux_3.mask_regions(directory_local+'Abell_85_I_plane_freq1.fits', region_name,'/data2/goesaert/testmask.fits', maskoutside=False)

In [24]:
start = time.time()
source_num = 1#len(rsl)
flux_measurements_I = np.zeros((12, source_num))
flux_measurements_Q = np.zeros((12, source_num))
flux_measurements_U = np.zeros((12, source_num))

for i in range(12):
    hdul_image = fits.open(directory_univ_I+'Abell_85_I_plane_freq'+(2-len(str((i+1))))*'0'+str(i+1)+'.fits')
    for n in tqdm.tqdm(range(source_num)):#range(len(rsl))):
        measurement = flux_py3.integratedflux(directory_local+'Abell_85_I_plane_freq'+(2-len(str((i+1))))*'0'+str(i+1)+'.fits'\
                                          , reg_file, n, hdul=hdul_image, test=True)
        flux_measurements_I[i,n] = measurement
    hdul_image.close()
print('I is done.')

for i in range(12):
    hdul_image = fits.open(directory_univ_Q+'Abell_85_Q_plane_freq'+(2-len(str((i+1))))*'0'+str(i+1)+'.fits')
    for n in tqdm.tqdm(range(source_num)):#range(len(rsl))):
        measurement = flux_py3.integratedflux(directory_local+'Abell_85_Q_plane_freq'+(2-len(str((i+1))))*'0'+str(i+1)+'.fits'\
                                          , reg_file, n, hdul=hdul_image, test=True)
        flux_measurements_Q[i,n] = measurement
    hdul_image.close()
print('Q is done.')

for i in range(12):
    hdul_image = fits.open(directory_univ_U+'Abell_85_U_plane_freq'+(2-len(str((i+1))))*'0'+str(i+1)+'.fits')
    for n in tqdm.tqdm(range(source_num)):#range(len(rsl))):
        measurement = flux_py3.integratedflux(directory_local+'Abell_85_U_plane_freq'+(2-len(str((i+1))))*'0'+str(i+1)+'.fits'\
                                          , reg_file, n, hdul=hdul_image, test=True)
        flux_measurements_U[i,n] = measurement
    hdul_image.close()
print('U is done.')

end = time.time()
print("The time of execution of above program is :", end-start)

#savedir =r'/net/vdesk/data2/GoesaertW/Meerkat_Data/Abell_85/Photometry_BDSF/'

#np.save(savedir+'Abell85_BDSF_I.npy', flux_measurements_I)
#np.save(savedir+'Abell85_BDSF_Q.npy', flux_measurements_Q)
#np.save(savedir+'Abell85_BDSF_U.npy', flux_measurements_U)

  0%|          | 0/1 [00:00<?, ?it/s]


ValueError: Number of world inputs (1) does not match expected (3)